In [2]:
import yfinance as yf
import pandas as pd

# =========================
# CONFIG
# =========================
stock_symbol = "TSLA"   # change if needed
ticker = yf.Ticker(stock_symbol)

# =========================
# FETCH OPTION DATA
# =========================
expiry = ticker.options[0]
option_chain = ticker.option_chain(expiry)
calls = option_chain.calls

# Fix NaN volume
calls['volume'] = calls['volume'].fillna(0)

# Current stock price
current_price = ticker.history(period="1d")['Close'].iloc[-1]
print("Current Stock Price:", current_price)

# =========================
# FILTER ATM ZONE
# =========================
atm_calls = calls[
    (calls['strike'] >= current_price - 50) &
    (calls['strike'] <= current_price + 50)
].copy()

# =========================
# GAMMA SCORE LOGIC
# =========================
# Simple fresher logic:
# Gamma Score = Volume * (Open Interest + 1)

atm_calls['gamma_score'] = atm_calls['volume'] * (atm_calls['openInterest'] + 1)

# =========================
# SORT & RANK
# =========================
gamma_ranked = atm_calls.sort_values(by='gamma_score', ascending=False)

print("\n===== GAMMA SCORE RANKING =====")
print(gamma_ranked[['strike', 'volume', 'openInterest', 'gamma_score']].head(10))


Current Stock Price: 437.5

===== GAMMA SCORE RANKING =====
    strike   volume  openInterest  gamma_score
59   430.0  76778.0             0      76778.0
63   440.0  44166.0             0      44166.0
65   445.0  35621.0             0      35621.0
67   450.0  32555.0             0      32555.0
61   435.0  24033.0             0      24033.0
64   442.5  17172.0             0      17172.0
71   460.0  16912.0             0      16912.0
66   447.5  16257.0             0      16257.0
75   470.0  14441.0             0      14441.0
74   467.5  12392.0             0      12392.0
